In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads

### Build tables

In [ ]:
query ="""
-- Create AIS table
CREATE OR REPLACE EXTERNAL TABLE `wsdemo-457314.ais.fullais`
OPTIONS (
  format = 'PARQUET', -- Change to the appropriate file format (e.g., JSON, PARQUET, etc.)
  uris = ['gs://fullaisdata/AIS/*']
);

-- Add h3 columns

ALTER TABLE `wsdemo-457314.ais.fullais`
ADD COLUMN h3_lv2 STRING,
ADD COLUMN h3_lv3 STRING,
ADD COLUMN h3_lv4 STRING,
ADD COLUMN h3_lv5 STRING,
ADD COLUMN h3_lv6 STRING,
ADD COLUMN h3_lv7 STRING;
UPDATE `wsdemo-457314.ais.fullais`
SET
  h3_lv7 = carto.H3_FROMGEOGPOINT(ST_GEOGFROMTEXT(geometry), 7)
WHERE geometry IS NOT NULL;
UPDATE `wsdemo-457314.ais.fullais`
SET
  h3_lv2 = carto.H3_TOPARENT(h3_lv7, 2),
  h3_lv3 = carto.H3_TOPARENT(h3_lv7, 3),
  h3_lv4 = carto.H3_TOPARENT(h3_lv7, 4),
  h3_lv5 = carto.H3_TOPARENT(h3_lv7, 5),
  h3_lv6 = carto.H3_TOPARENT(h3_lv7, 6)
WHERE h3_lv7 IS NOT NULL;

-- Create shadowtanker table
CREATE OR REPLACE TABLE `wsdemo-457314.ais.shadowtankers` AS
SELECT *
FROM `wsdemo-457314.ais.fullais`
WHERE mmsi IN (352002786,
626311000,
273257910,
626367000,
626364000,
636022550,
352003372,
636014191,
626393000,
626388000,
626378000,
626379000,
626382000,
352002202,
626369000,
626362000,
626390000,
626377000,
613702404,
511100566,
352003308,
355070000,
351062000,
352421000,
248330000,
372119000,
352001387,
352001485,
352001485,
352001509,
636019831,
621819061,
352002898,
626293000,
626290000,
511101196,
636022240,
620999315,
312038000,
620999378,
312513000,
626383000,
273611590,
621819076,
477007600,
511101363,
352002109,
636023569,
357654000,
538005802,
273216520,
352002074,
626361000,
352002195,
352003367,
626421000,
626391000,
626389000,
352003366,
352003364,
352002194,
626408000,
626365000,
626375000,
626371000,
626415000,
626416000,
626370000,
352002203,
626292000) OR imo in (9302970,
9307815,
9306782,
9256078,
9270529,
9322267,
9610808,
9413559,
9339313,
9341093,
9411020,
9341067,
9312884,
9249128,
9256054,
9610793,
9312896,
9412359,
9293741,
9194995,
9402471,
9258521,
9539676,
9230971,
9395317,
9809356,
9235713,
9310159,
9310159,
9310147,
9269075,
9237072,
9378632,
9288746,
9402732,
9434890,
9236353,
9281683,
9256468,
9259185,
9282522,
9306794,
9866392,
9230969,
9315082,
9224439,
9237785,
9311751,
9271951,
9697210,
9156993,
9282508,
9276030,
9301380,
9309576,
9333412,
9339325,
9341079,
9354301,
9354313,
9360128,
9397559,
9412347,
9413561,
9413573,
9577082,
9577094,
9610781,
9901025,
9290335);

-- Parse timestamp
ALTER TABLE `wsdemo-457314.ais.fullais`
ADD COLUMN ts TIMESTAMP;
UPDATE `wsdemo-457314.ais.fullais`
SET ts = TIMESTAMP(dt)
WHERE dt IS NOT NULL;

-- Calculate ship associations (number of times moving in the same level6 h3 hex in the same hour)
CREATE OR REPLACE TABLE `wsdemo-457314.ais.ship_associations` AS
WITH filtered AS (
  SELECT
    mmsi,
    TIMESTAMP_TRUNC(TIMESTAMP(dt), HOUR) AS ts_hour,
    h3_lv6
  FROM `wsdemo-457314.ais.fullais`
  WHERE sog > 0
    AND h3_lv6 IS NOT NULL
    AND dt IS NOT NULL
),
vessel_pairs AS (
  SELECT
    a.mmsi AS mmsi1,
    b.mmsi AS mmsi2
  FROM filtered a
  JOIN filtered b
    ON a.ts_hour = b.ts_hour
   AND a.h3_lv6 = b.h3_lv6
   AND a.mmsi < b.mmsi
)
SELECT
  mmsi1,
  mmsi2,
  COUNT(*) AS co_occurrence_count
FROM vessel_pairs
GROUP BY mmsi1, mmsi2
ORDER BY co_occurrence_count DESC;

-- Calculate STATIONARY ship associations (number of times in the same level6 h3 hex in the same hour)
CREATE OR REPLACE TABLE `wsdemo-457314.ais.ship_associations` AS
WITH filtered AS (
  SELECT
    mmsi,
    TIMESTAMP_TRUNC(TIMESTAMP(dt), HOUR) AS ts_hour,
    h3_lv6
  FROM `wsdemo-457314.ais.fullais`
  WHERE h3_lv6 IS NOT NULL
    AND dt IS NOT NULL
),
vessel_pairs AS (
  SELECT
    a.mmsi AS mmsi1,
    b.mmsi AS mmsi2
  FROM filtered a
  JOIN filtered b
    ON a.ts_hour = b.ts_hour
   AND a.h3_lv6 = b.h3_lv6
   AND a.mmsi < b.mmsi
)
SELECT
  mmsi1,
  mmsi2,
  COUNT(*) AS co_occurrence_count_stationary
FROM vessel_pairs
GROUP BY mmsi1, mmsi2
ORDER BY co_occurrence_count DESC;


-- We only care about shadowtanker associations
CREATE TABLE wsdemo-457314.ais.shadowtanker_ship_associations AS
SELECT *
FROM wsdemo-457314.ais.ship_associations
WHERE mmsi1 IN (SELECT DISTINCT mmsi FROM wsdemo-457314.ais.shadowtankers)
   OR mmsi2 IN (SELECT DISTINCT mmsi FROM wsdemo-457314.ais.shadowtankers);


-- ############ Use k-means to detect shadowtanker movement anomalies
-- create features table
CREATE OR REPLACE TABLE `wsdemo-457314.ais.fullais_features` AS
SELECT
  mmsi,
  vessel_class,
  TIMESTAMP_TRUNC(TIMESTAMP(dt), HOUR) AS ts_hour,
  sog                      AS speed,
  COS(heading * 3.14159 / 180) AS hdg_x,
  SIN(heading * 3.14159 / 180) AS hdg_y,
  SAFE_CAST(
    FARM_FINGERPRINT(h3_lv5) AS FLOAT64
  )                        AS h3_hash
FROM `wsdemo-457314.ais.fullais`
WHERE sog      > 0
  AND heading  IS NOT NULL
  AND h3_lv5   IS NOT NULL
  AND dt        IS NOT NULL;

--Train k-means

CREATE OR REPLACE MODEL `wsdemo-457314.ais.fullais_kmeans`
OPTIONS(
  model_type       = 'kmeans',
  num_clusters     = 8,           
  standardize_features = TRUE
) AS
SELECT
  speed,
  hdg_x,
  hdg_y,
  h3_hash
FROM `wsdemo-457314.ais.fullais_features`;

--Use model to detect shadowtanker movement anomalies

CREATE OR REPLACE TABLE `wsdemo-457314.ais.shadowtankers_anomalies` AS
WITH features AS (
  SELECT
    f.mmsi,
    f.dt,
    f.h3_lv5,
    f.sog                                AS speed,
    COS(f.heading * 3.14159 / 180)              AS hdg_x,
    SIN(f.heading * 3.14159 / 180)              AS hdg_y,
    SAFE_CAST(FARM_FINGERPRINT(f.h3_lv5) AS FLOAT64) AS h3_hash
  FROM `wsdemo-457314.ais.fullais` AS f
  JOIN `wsdemo-457314.ais.shadowtankers` AS s
    ON f.mmsi = s.mmsi
  WHERE f.sog > 0
    AND f.heading IS NOT NULL
    AND f.h3_lv5 IS NOT NULL
    AND f.dt IS NOT NULL
)
SELECT
  mmsi,
  dt,
  h3_lv5,
  is_anomaly,
  normalized_distance
FROM ML.DETECT_ANOMALIES(
  MODEL `wsdemo-457314.ais.fullais_kmeans`,
  STRUCT(0.01 AS contamination),
  TABLE features
)
WHERE is_anomaly = TRUE
ORDER BY normalized_distance DESC;

"""

# Run the query
from google.cloud import bigquery
client = bigquery.Client()
table_id = "wsdemo-457314.ais.shadowtankers"
query_job = client.query(query)


### Load data

In [ ]:
#### LOAD DATA DIRECTLY #####
query = f"SELECT * FROM `{table_id}`"
df = client.query(query).to_dataframe()
df.to_csv("../data/shadowtankers.csv", index=False)

In [ ]:
##### LOAD DATA FROM CSV #####
df = pd.read_csv("../data/shadowtankers.csv")

### Initial data exploration

In [ ]:
df.info()

#### Is each IMO associated with just one MMSI (and vice versa)?

In [ ]:
# Check for 1:1 match for imo to mmsi
imo_to_mmsi_unique = df.groupby('imo')['mmsi'].nunique()
mmsi_to_imo_unique = df.groupby('mmsi')['imo'].nunique()

# Determine if there are any imo with multiple mmsi or vice versa
is_imo_to_mmsi_one_to_one = (imo_to_mmsi_unique.max() == 1)
is_mmsi_to_imo_one_to_one = (mmsi_to_imo_unique.max() == 1)

# Final result
if is_imo_to_mmsi_one_to_one and is_mmsi_to_imo_one_to_one:
    print("Yes, there is a 1:1 match between imo and mmsi.")
else:
    print("No, there is not a 1:1 match between imo and mmsi.")